1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
1. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 
1. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 
1. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 
1. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 
1. What topics are grads continuing to reference after graduation and into their jobs (for each program)? 
1. Which lessons are least accessed? 
1. Anything else I should be aware of? 

* What topics are more difficult than others
* Lessons being accessed the least - maybe the lessons need to be re-written or the lesson is simple. If a lesson that is not being used at all, maybe there are lessons that replace them or they need to be merged
* Maybe there is a particular lesson a particular cohort accessed more than the others. Needed to understand how instructors are doing
* Find outliers to determine if there is webscraping happening on the lessons or not.
  * IPs
  * User agents
  * Odd times 
* Date range for when the shut off was successful
* What content are grads are looking at into their jobs


In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

print("Initalized")

Initalized


In [2]:
# Read the data
data = pd.read_csv('anonymized-curriculum-access.txt', sep=" ", header=None, na_values='"-"')
# From the https://ds.codeup.com/anomaly-detection/time-series-anomaly-detection-part-3/ rename columns as needed
data.columns=['date', 'time', 'page_viewed','user_id','cohort_id','ip']
# Combine the date time columns and convert it to a datetimeobj
data['datetime'] = data['date'] + ' ' + data['time']
data['datetime'] = pd.to_datetime(data.datetime)
# Create a columns for hour, day name, and month
data['hour'] = data['datetime'].dt.hour
data['day'] = data['datetime'].dt.day_name()
data['month'] = data['datetime'].dt.month_name()
#Set the index to the new datetime column and sort it
data

,date,time,page_viewed,user_id,cohort_id,ip,datetime,hour,day,month
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2018-01-26 09:55:03,9,Friday,January
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2018-01-26 09:56:02,9,Friday,January
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2018-01-26 09:56:05,9,Friday,January
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2018-01-26 09:56:06,9,Friday,January
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-26 09:56:24,9,Friday,January
...,...,...,...,...,...,...,...,...,...,...
719454,2020-11-02,16:48:13,javascript-i/coffee-project,763,62.0,107.192.148.199,2020-11-02 16:48:13,16,Monday,November
719455,2020-11-02,16:48:17,javascript-i/mapbox-api,771,62.0,172.125.226.175,2020-11-02 16:48:17,16,Monday,November
719456,2020-11-02,16:48:18,javascript-i/coffee-project,771,62.0,172.125.226.175,2020-11-02 16:48:18,16,Monday,November
719457,2020-11-02,16:48:28,javascript-i/bom-and-dom/bom,771,62.0,172.125.226.175,2020-11-02 16:48:28,16,Monday,November


In [3]:
# Read the cohort data Zach gave us into a Df
cohort = pd.read_csv('cohorts.csv')
# Merge the cohort data onto usage df 
data = data.merge(cohort, on='cohort_id', how='left')
#print the head
data.shape

(719459, 14)

In [4]:
# Create an empty column to hold classtype 
data['classtype'] = 'x'
# Use .loc and column name to create classtype values from class id
data.loc[data['program_id'] == 1, 'classtype'] = 'Web-SanAntonio'
data.loc[data['program_id'] == 2, 'classtype'] = 'Web-Dallas'
data.loc[data['program_id'] == 3, 'classtype'] = 'DataScience-SanAntonio'
data.loc[data['program_id'] == 4, 'classtype'] = 'Web-Houston'
#Then split the class type column into two on -
data[['program','location']] = data.classtype.str.split("-",expand=True) 
# Base data frame created 
data

,date,time,page_viewed,user_id,cohort_id,ip,datetime,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2018-01-26 09:55:03,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2018-01-26 09:56:02,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2018-01-26 09:56:05,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2018-01-26 09:56:06,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-26 09:56:24,9,Friday,January,Teddy,2018-01-08,2018-05-17,2.0,Web-Dallas,Web,Dallas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719454,2020-11-02,16:48:13,javascript-i/coffee-project,763,62.0,107.192.148.199,2020-11-02 16:48:13,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas
719455,2020-11-02,16:48:17,javascript-i/mapbox-api,771,62.0,172.125.226.175,2020-11-02 16:48:17,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas
719456,2020-11-02,16:48:18,javascript-i/coffee-project,771,62.0,172.125.226.175,2020-11-02 16:48:18,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas
719457,2020-11-02,16:48:28,javascript-i/bom-and-dom/bom,771,62.0,172.125.226.175,2020-11-02 16:48:28,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas


In [5]:
data.isnull().sum()

date               0
time               0
page_viewed        1
user_id            0
cohort_id      44840
ip                 0
datetime           0
hour               0
day                0
month              0
name           44840
start_date     44840
end_date       44840
program_id     44840
classtype          0
program            0
location       44840
dtype: int64

In [6]:
# Create seperate data fames for web and datascience
web = data[data['program']=='Web']
ds = data[data['program']=='DataScience']
# sort the values by datetime
ds = ds.sort_values(by='datetime')
web = web.sort_values(by='datetime')
# set the index to datetime
ds=ds.set_index('datetime')
web=web.set_index('datetime')
ds

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2019-08-20 09:39:58,2019-08-20,09:39:58,/,466,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2019-08-20 09:39:59,2019-08-20,09:39:59,/,467,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2019-08-20 09:39:59,2019-08-20,09:39:59,/,468,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2019-08-20 09:40:02,2019-08-20,09:40:02,/,469,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2019-08-20 09:40:08,2019-08-20,09:40:08,/,470,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-02 15:50:07,2020-11-02,15:50:07,appendix/open_data,698,59.0,24.28.150.196,15,Monday,November,Darden,2020-07-13,2021-01-12,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2020-11-02 15:52:23,2020-11-02,15:52:23,timeseries/sarimax,698,59.0,24.28.150.196,15,Monday,November,Darden,2020-07-13,2021-01-12,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2020-11-02 15:57:22,2020-11-02,15:57:22,/,616,55.0,70.114.9.241,15,Monday,November,Curie,2020-02-03,2020-07-07,3.0,DataScience-SanAntonio,DataScience,SanAntonio


In [7]:
# resample the index by moth
ds_month = ds.groupby(['name', 'page_viewed']).resample('M').size().reset_index().set_index('datetime')
web_month = web.groupby(['name', 'page_viewed']).resample('M').size().reset_index().set_index('datetime')
ds_month.columns=['name', 'page', 'hits']
web_month.columns=['name', 'page', 'hits']
web_month

,name,page,hits
datetime,,,
2019-03-31,Andromeda,/,132
2019-04-30,Andromeda,/,232
2019-05-31,Andromeda,/,188
2019-06-30,Andromeda,/,110
2019-07-31,Andromeda,/,108
...,...,...,...
2019-07-31,Zion,uploads/598dc43df39e2.jpg,1
2019-12-31,Zion,web-design/intro,1
2019-07-31,Zion,web-design/ui/visuals,1


In [92]:
# group by cohort name and page to and mean to get the total number of hits per page per cohort
ds_avghits = ds_month.groupby(['name', 'page']).sum().unstack().reset_index(col_level=1)
ds_avghits.columns = ds_avghits.columns.droplevel(0)
ds_avghits = ds_avghits.reset_index()
ds_avghits = ds_avghits.transpose()
ds_avghits = ds_avghits.drop(labels=['index', 'name'])
ds_avghits = ds_avghits.reset_index().set_index('page', drop=True)
ds_columns=ds_month.name.unique()
ds_avghits.columns=ds_columns
ds_avghits = ds_avghits.fillna(0)
ds_avghits['total_hits'] = ds_avghits.sum(axis=1)
ds_avghits['total_avg'] = ds_avghits.mean(axis=1)
ds_avghits[ds_avghits['total_avg'] >= 300]

,Bayes,Curie,Darden,total_hits,total_avg
page,,,,,
/,1842.0,1523.0,2041.0,5406.0,2703.0
1-fundamentals/1.1-intro-to-data-science,615.0,461.0,460.0,1536.0,768.0
1-fundamentals/1.2-data-science-pipeline,221.0,146.0,271.0,638.0,319.0
1-fundamentals/AI-ML-DL-timeline.jpg,624.0,465.0,470.0,1559.0,779.5
1-fundamentals/DataToAction_v2.jpg,230.0,148.0,284.0,662.0,331.0
1-fundamentals/modern-data-scientist.jpg,626.0,467.0,470.0,1563.0,781.5
10-anomaly-detection/1-overview,383.0,345.0,14.0,742.0,371.0
10-anomaly-detection/AnomalyDetectionCartoon.jpeg,386.0,345.0,14.0,745.0,372.5
3-sql/1-mysql-overview,142.0,441.0,121.0,704.0,352.0


In [91]:
# group by cohort name and page to and mean to get the total number of hits per page per cohort
web_avghits = web_month.groupby(['name', 'page']).sum().unstack().reset_index(col_level=1)
web_avghits.columns = web_avghits.columns.droplevel(0)
web_avghits = web_avghits.reset_index()
web_avghits = web_avghits.transpose()
web_avghits = web_avghits.drop(labels=['index', 'name'])
web_avghits = web_avghits.reset_index().set_index('page', drop=True)
web_columns=web_month.name.unique()
web_avghits.columns=web_columns
web_avghits = web_avghits.fillna(0)
web_avghits['total_hits'] = web_avghits.sum(axis=1)
web_avghits['total_avg'] = web_avghits.mean(axis=1)
web_avghits[web_avghits['total_avg'] >= 300]

,Andromeda,Apex,Apollo,Arches,Badlands,Bash,Betelgeuse,Ceres,Deimos,Denali,...,Staff,Teddy,Ulysses,Voyageurs,Wrangell,Xanadu,Yosemite,Zion,total_hits,total_avg
page,,,,,,,,,,,,,,,,,,,,,
/,1156.0,1244.0,1.0,622.0,17.0,532.0,868.0,1620.0,1319.0,1.0,...,4633.0,1754.0,1618.0,2098.0,1112.0,916.0,962.0,1756.0,30668.0,1614.105263
appendix,161.0,426.0,0.0,133.0,0.0,80.0,180.0,408.0,287.0,0.0,...,989.0,488.0,174.0,393.0,209.0,320.0,142.0,289.0,6678.0,351.473684
html-css,372.0,708.0,0.0,215.0,2.0,214.0,425.0,753.0,609.0,0.0,...,977.0,225.0,555.0,528.0,405.0,587.0,362.0,649.0,10363.0,545.421053
java-i,336.0,528.0,0.0,132.0,1.0,157.0,422.0,510.0,434.0,0.0,...,624.0,501.0,375.0,639.0,401.0,483.0,245.0,593.0,8699.0,457.842105
java-ii,395.0,595.0,0.0,167.0,0.0,190.0,444.0,667.0,506.0,0.0,...,884.0,494.0,423.0,754.0,323.0,501.0,268.0,613.0,9956.0,524.000000
java-iii,430.0,616.0,0.0,153.0,0.0,72.0,444.0,674.0,517.0,0.0,...,908.0,712.0,411.0,770.0,335.0,495.0,340.0,742.0,10610.0,558.421053
javascript-i,509.0,610.0,0.0,294.0,1.0,319.0,686.0,977.0,696.0,0.0,...,1263.0,589.0,523.0,884.0,553.0,718.0,437.0,868.0,13902.0,731.684211
javascript-ii,290.0,431.0,0.0,204.0,0.0,175.0,374.0,519.0,392.0,0.0,...,744.0,468.0,308.0,584.0,278.0,493.0,332.0,637.0,8367.0,440.368421
jquery,335.0,437.0,0.0,133.0,0.0,233.0,540.0,632.0,466.0,0.0,...,615.0,498.0,363.0,583.0,374.0,543.0,278.0,559.0,8846.0,465.578947
